# Transfomer Model: Arabic to English Translation
* TYPE OF MODEL: Using a transformer model. Is significant advancement in sequence-to-sequence tasks especially for natural language processing. Uses a self attention mechanism that allows the model to focus on different parts of the input sequence, capturing long-range dependencies more effectively than traditionall RNNs and LSTMs. Additionally it supports parallel processing, whihc enhances training speed.


### 1. Importing Dependencies/Installing Libraries


In [2]:
!pip install torch==2.2.0 torchtext==0.16.2

  Using cached torch-2.2.0-cp312-cp312-win_amd64.whl.metadata (26 kB)
  Using cached torchtext-0.16.2-cp312-cp312-win_amd64.whl.metadata (7.5 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached tqdm-4.66.6-py3-none-any.whl.metadata (57 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached torchdata-0.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached charset_normalizer-3.4.0-cp312-cp312-win_amd64.whl.metadata (34 kB)
  Using cached idna-3.10-py3-none-any.whl.

In [3]:
!pip install nltk

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 11.4 MB/s eta 0:00:00


installed torch==2.0.1 and torchtext==0.15.2

In [7]:
!pip install numpy 1.26.0

ERROR: Could not find a version that satisfies the requirement 1.26.0 (from versions: none)
ERROR: No matching distribution found for 1.26.0


In [4]:
import torch # for building neural network and other functions
import torch.nn as nn
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader # used inplace of BucketIterator
import torch.optim as optim # optimizer
import nltk # for tokenization
from nltk.tokenize import word_tokenize
from datasets import load_dataset # Hugging Face Library to import datasets easily
from torch.utils.tensorboard import SummaryWriter
# import ipdb # for debugging
from torchtext.data.metrics import bleu_score
import numpy as np


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\John.Mathis.SERN\Desktop\SSE672\machine_translation\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\John.Mathis.SERN\Desktop\SSE672\machine_translation\.venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\John.Mathis.SERN\Desktop\SSE672\machine_translation\.venv\Lib\sit

ModuleNotFoundError: No module named 'datasets'

### Step 1. Data Preprocessing
* DATA PREPROCESSING: sourced out a dataset from Hugging Face and splitit into training and tests setsw. The toeknization process employs NLTK to break down Arabic and English text into manageable tokens. Once tokenized, the text is converted into numerical representations using custom built vocabularies, that facilitate the training of the transformer model.


In [71]:
# Import the dataset from Hugging Face
ds = load_dataset("mohamed-khalil/ATHAR")
print(ds)

DatasetDict({
    train: Dataset({
        features: ['arabic', 'english'],
        num_rows: 65043
    })
    test: Dataset({
        features: ['arabic', 'english'],
        num_rows: 1000
    })
})


In [46]:

# Assuming the dataset has 'train' and 'test' splits and you're interested in lowercasing the 'src' and 'trg' fields
def lowercase_text(example):
    example['arabic'] = example['arabic'].lower()  # Lowercase the source text
    example['english'] = example['english'].lower()  # Lowercase the target text
    return example

# Apply the lowercasing to the train and test sets
ds['train'] = ds['train'].map(lowercase_text)
ds['test'] = ds['test'].map(lowercase_text)



Map: 100%|██████████| 1000/1000 [00:00<00:00, 35094.67 examples/s]


In [72]:
# Split the data
# HuggingFace already has this split into a 'train' set and a 'test' set
full_train_dataset = ds['train']

# Split the train_dataset into training and validation sets (80% train, 20% validation)
train_split = full_train_dataset.train_test_split(test_size=0.2, seed=42)
# Access the new sets
train_data = train_split['train']
valid_data = train_split['test']
#test_data = ds['test']
test_data = ds['test']

print(f"Training set size3: {len(train_data)}")
print(f"Validation set size3: {len(valid_data)}")
print(f"Test set size: {len(test_data)}")

Training set size3: 52034
Validation set size3: 13009
Test set size: 1000


In [73]:
"""Defining Tokenization functions"""
nltk.download('punkt_tab') # downloads the language models for tokenizing

# functions for tokenizing using nltk library; not necessary since word_tokenize() is a nice compact function, but lets you at least renname it
def tokenize_arabic(text):
    if isinstance(text, bytes):
        text = text.decode('utf-8')  # Adjust encoding as needed
    return word_tokenize(text)

def tokenize_eng(text):
    if isinstance(text, bytes):
        text = text.decode('utf-8')  # Adjust encoding as needed
    return word_tokenize(text)

[nltk_data] Downloading package punkt_tab to C:\Users\John
[nltk_data]     Wesley\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [74]:
# Block for Testing Tokenization
arabic_text = "مرحبا بالعالم! هذا هو اختباري الأول."
arabic_tokens = tokenize_arabic(arabic_text)
print("Arabic tokens: ", arabic_tokens)

english_text = "Hello world! This is my first test."
english_tokens = tokenize_eng(english_text)
print("English tokens: ", english_tokens)

Arabic tokens:  ['مرحبا', 'بالعالم', '!', 'هذا', 'هو', 'اختباري', 'الأول', '.']
English tokens:  ['Hello', 'world', '!', 'This', 'is', 'my', 'first', 'test', '.']


In [75]:
"""Preprocessing Functions"""
# Convert text to lowercase, tokenize it and add <sos> and <eos> to tokens
def process_text(text, tokenizer):
  if isinstance(text, bytes):  # Check if text is in bytes
      text = text.decode('utf-8')  # Decode bytes to string
  tokens = tokenizer(text)
  tokens = ["<sos>"] + tokens + ["<eos>"]
  return tokens

# yields processed tokens for each text in the dataset
# is a generator that will pull one sentence at a time from dataset and tokenize the sentence
def yield_tokens(data_iter, tokenizer):
  for text in data_iter:
    yield process_text(text, tokenizer)

# Converts tokens into numerical representations using a vocab
def numericalize(text, tokenizer, vocab):
  if isinstance(text, bytes):
    text = text.decode('utf-8')
  tokens = process_text(text, tokenizer)
  # return [vocab[token] for token in tokens]
  return [vocab[token] if token in vocab else vocab["<unk>"] for token in tokens]

In [76]:
# Example using the above functions
mock_vocab = {
    "<pad>": 0,
    "<sos>": 1,
    "<eos>": 2,
    "<unk>": 3,
    "أحب": 4,
    "تعلم": 5,
    "البرمجة": 6
}
text = "أحب تعلم البرمجة"
numericalized_text = numericalize(text, tokenize_arabic, mock_vocab)
print(numericalized_text)  # [token indices]

[1, 4, 5, 6, 2]


In [77]:
"""Create vocabularies"""
# Prepare token iterators
train_arabic_tokens = yield_tokens((example['arabic'] for example in train_data), tokenize_arabic)
train_english_tokens = yield_tokens((example['english'] for example in train_data), tokenize_eng)

# Build vocabularies
# build_vocab_from_iterator takes in  iterator used to build vocab, min_freq needed to include token in the vocab, special symbols to add to vocab, and max tokens to create vocab from most frequent tokens
arabic_vocab = build_vocab_from_iterator(train_arabic_tokens, min_freq=2, specials=["<pad>", "<sos>", "<eos>", "<unk>"], max_tokens=10000)
english_vocab = build_vocab_from_iterator(train_english_tokens, min_freq=2, specials=["<pad>", "<sos>", "<eos>", "<unk>"], max_tokens=10000)

# Set default index for padding token
# will retrieve the index associated with the <unk> token
# set_default_index is a torchtext method to define what index to use if a word is not in the vocab (aka an out-of-vocabulary token (OOV))
# any unkonwn tokens will map to the "<unk>" index
arabic_vocab.set_default_index(arabic_vocab["<unk>"])
english_vocab.set_default_index(english_vocab["<unk>"])

src_pad_idx = arabic_vocab["<pad>"] # Get <pad> token indices from both vocabs
trg_pad_idx = english_vocab["<pad>"]
print(src_pad_idx)
print(trg_pad_idx)
print(arabic_vocab["<unk>"])
print(english_vocab["<unk>"])
print(arabic_vocab["<sos>"])
print(english_vocab["<sos>"])
print(arabic_vocab["<eos>"])
print(english_vocab["<eos>"])


0
0
3
3
1
1
2
2


In [78]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
# Custom class and functions used instead of outdated Field and BucketIterator
# Class is used to fetch, batch, and iterate over the tokenized, numericalized data during training.
# The DataLoader also provides convenient batching and shuffling, which are crucial for efficient model training.
class TranslationDataset(Dataset):
  def __init__(self, data, arabic_vocab, english_vocab, max_len=100):
    self.data = data
    self.arabic_vocab = arabic_vocab
    self.english_vocab = english_vocab
    self.max_len = max_len

  def __len__(self):
    return len(self.data)

  # Method to fetch both the arabic and english sentence pairs at the idx-th index
  def __getitem__(self, idx):
    arabic_text = self.data[idx]['arabic']
    english_text = self.data[idx]['english']

    # Numericalize the text
    arabic_numericalized = numericalize(arabic_text, tokenize_arabic, self.arabic_vocab)
    english_numericalized = numericalize(english_text, tokenize_eng, self.english_vocab)

    # Truncate if exceeding max_len
    arabic_numericalized = arabic_numericalized[:self.max_len]
    english_numericalized = english_numericalized[:self.max_len]
    return torch.tensor(arabic_numericalized), torch.tensor(english_numericalized)

def collate_fn(batch):
    src_batch, trg_batch = zip(*batch)
    # Pad sequences within the batch to the max length of the batch sequences
    src_batch = pad_sequence(src_batch, padding_value=src_pad_idx, batch_first=False)
    trg_batch = pad_sequence(trg_batch, padding_value=trg_pad_idx, batch_first=False)
    """DEBUG"""
    # print(f"Padded src_batch: {src_batch}")
    # print(f"Padded trg_batch: {trg_batch}")

    class Batch:
      def __init__(self, src, trg):
        self.src = src
        self.trg = trg

    batch = Batch(src_batch, trg_batch)
    return batch

train_dataset = TranslationDataset(train_data, arabic_vocab, english_vocab, max_len=100)
valid_dataset = TranslationDataset(valid_data, arabic_vocab, english_vocab, max_len=100)
test_dataset = TranslationDataset(test_data, arabic_vocab, english_vocab, max_len=100)


* MODEL ARCHITECTURE: Features separate embedding layers for both arabic and english inputs. Position embeddings are added to account for the order of tokens wihthin the sequence. The transformer comprises six encoder and six decoder layers, each utilizing eight attention heads for robust feature extraction. To combat overfitting, dropout is implemented throughout the model, and the output layyer is a linear layer that maps the transformer's outputs to the target vocabulary for english translation. This particular architecture is based on the paper Attention is All you Need.


In [79]:
import torch.nn.functional as F
"""Defining Transformer Model"""
class Transformer(nn.Module):
  # Constructor for Transformer class
    def __init__(
        self,
        embedding_size,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        num_heads, # number of heads used for attention mechanism
        num_encoder_layers, # number of encoder layers
        num_decoder_layers, # number of decoder layers
        forward_expansion,
        dropout,
        max_len,
        device
    ):
        # Defining neural network layers
        super(Transformer, self).__init__()

        """Creates an embedding layer for the source(arabic) vocabulary
           initializes a lookup table that maps each word index (from source vocab) to a dense vector representation of fixed size(aka embedding size)
           src_vocab_size is specifying the totla number of unique words (tokens) in the source vocab
           embedding_size specifies the size of the vector for each word; e.g. 256, then each word will be represented by a 256-dimensional vector"""

        self.src_word_embedding = nn.Embedding(src_vocab_size+1, embedding_size)

        """ Createas embedding layer for positional information
            max_len is the length of the sequences the model will handle
            each position in a sequence (from 0 to max_len -1) gets a unique vector representation of size embedding_size
            Transformers do not consider the iinput sequences (processes all tokens at once) positional embeddings are crucial for inejcting information
            about the order of the tokens helping the model understand the sequence structure to learn relationships based on positions within a sentence"""

        self.src_position_embedding = nn.Embedding(max_len, embedding_size)
        self.trg_word_embedding = nn.Embedding(trg_vocab_size+1, embedding_size)
        self.trg_position_embedding = nn.Embedding(max_len, embedding_size)
        self.device = device # set device
        # Using PyTorch implementation of transformer
        self.transformer = nn.Transformer(
            embedding_size,
            num_heads,
            num_encoder_layers,
            num_decoder_layers,
            forward_expansion,
            dropout,
        )

        # Fully connected (linear) layer
        # Used to transform the output of the model (After processing through the network) into a format suitable for generating predictions for the target vocab
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout) # randomly zeros some of the elements of the input tensor; this is to prevent overfitting during training by randomly dropping (set to zero) a sepcified fraction of the neurons
        self.src_pad_idx = src_pad_idx # just assigning the index for the padding token

    def make_src_mask(self, src):
        """Purpose: So the attention mechanism knows which tokens to ignore (i.e. padding and unkown tokens) when computing attention scores. Ensures only meanignful tokens contribute to the output
          src shape: (src_len, N)
          src_len is the length of the source sequences
          N is the batch size (number of sequences processed in parallel)
          transpose swaps the first dimension with the second dimension
          creates a boolean mask: True if src is equal to the padding index """
        src_mask = src.transpose(0, 1) == self.src_pad_idx
        return src_mask # output for Pytorch (N, src_len)

    def forward(self, src, trg):
          # Ensure input is of the correct type
        assert src.dtype == torch.long, "Input src should be of type long."
        assert trg.dtype == torch.long, "Input trg should be of type long."

        # Debug output
        # print("Source Input Shape:", src.shape)
        # print("Target Input Shape:", trg.shape)

        # Validate indices
        assert src.max() < self.src_word_embedding.num_embeddings, f"Source indices out of range: {src.max().item()}"
        assert trg.max() < self.trg_word_embedding.num_embeddings, f"Target indices out of range: {trg.max().item()}"

        # Get dimensions
        src_seq_length, batch_size = src.shape
        trg_seq_length, _ = trg.shape

        src_positions = (
            # Creaates 1D tensor of integers ranging from 0 to src_seq_length -1.
            torch.arange(0, src_seq_length) # i.e src_seq_length = 5 => [0, 1, 2, 3, 4, 5]
            .unsqueeze(1) # transforms the tensor from 1D to 2D (adds dimension) [[0,1,2,3,4,5]]
            .expand(src_seq_length, batch_size) # replicates the unsqueezed tensor to match the desired shape; each position value will be repeated "trg_seq-length" times across the new dimension and batch_size times across the other dimension
            .to(self.device) # moves the tensor to the specified device (CPU or GPU)
        )

        trg_positions = (
            torch.arange(0, trg_seq_length)
            .unsqueeze(1)
            .expand(trg_seq_length, batch_size)
            .to(self.device)
        )

        # Compute embeddings
        #
        embed_src = self.dropout(
            (self.src_word_embedding(src) + self.src_position_embedding(src_positions))
        )

        embed_trg = self.dropout(
            (self.trg_word_embedding(trg) + self.trg_position_embedding(trg_positions))
        )

        src_padding_mask = self.make_src_mask(src)
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(self.device)

        out = self.transformer(
            embed_src,
            embed_trg,
            src_key_padding_mask=src_padding_mask,
            tgt_mask=trg_mask
        )

        out = self.fc_out(out)

        return out


In [80]:
def translate_sentence(model, sentence, arabic_vocab, english_vocab, device, max_length=100):
    # Set the model to evaluation mode
    model.eval()
    # Tokenize the input sentence
    src_indices = numericalize(sentence, tokenize_arabic, arabic_vocab)
    print(f"Source: {src_indices}")
    src_tensor = torch.tensor(src_indices).unsqueeze(1).to(device)

    trg_indices = [english_vocab["<sos>"]]
    trg_tensor = torch.tensor(trg_indices).unsqueeze(1).to(device)

    adjusted_max_length = min(max_length, len(trg_indices)*2)
    for _ in range(adjusted_max_length):

      with torch.no_grad():
        output = model(src_tensor, trg_tensor)
      output_token = output.argmax(2)[-1, -1].item()
      trg_indices.append(output_token)

      if output_token  == english_vocab["<eos>"]:
        break
    print(f"Target: {trg_indices}")
    translated_tokens = [english_vocab.get_itos()[idx] for idx in trg_indices[1:]]

    return ' '.join(translated_tokens)



In [81]:
print("Sample numericalized source:", numericalize("some Arabic text", tokenize_arabic, arabic_vocab))
print("Sample numericalized target:", numericalize("some English text", tokenize_eng, english_vocab))


Sample numericalized source: [1, 3, 3, 3, 2]
Sample numericalized target: [1, 103, 3, 2408, 2]


In [17]:
# def bleu(model, test_data, arabic_vocab, english_vocab, device):
#     total_score = 0
#     for i in range(len(test_data)):
#         # Debugging: Check that `test_data[i].trg` and `test_data[i].src` are accessible
#         print(f"Processing sentence {i+1}/{len(test_data)}")

#         # Get the target translation for the current sample
#         try:
#             reference_indices = numericalize(test_data[i].trg, tokenize_eng, english_vocab)
#             reference = [english_vocab.get_itos()[idx] for idx in reference_indices]
#         except KeyError as e:
#             print(f"KeyError for reference: {e}")
#             continue

#         # Generate a translation using your model
#         try:
#             translation_indices = numericalize(translate_sentence(model, test_data[i].src, english_vocab, arabic_vocab, device), tokenize_eng, english_vocab)
#             translation = [english_vocab.get_itos()[idx] for idx in translation_indices]
#         except KeyError as e:
#             print(f"KeyError for translation: {e}")
#             continue

#         # Calculate BLEU score for the current sample
#         score = bleu_score([reference], translation)

#         # Accumulate the scores
#         total_score += score

#     # Return the average score
#     return total_score / len(test_data) if len(test_data) > 0 else 0


In [82]:
def bleu(model, test_iterator, arabic_vocab, english_vocab, device):
    total_score = 0
    total_samples = 0

    for batch in test_iterator:
        src_batch, trg_batch = batch.src, batch.trg
        print(f"Processing batch with source shape {src_batch.shape} and target shape {trg_batch.shape}")

        for i in range(trg_batch.size(0)):  # Iterate through each example in the batch
            # Get the target translation for the current sample
            reference_indices = numericalize(trg_batch[i].cpu().numpy(), tokenize_eng, english_vocab)  # Make sure to convert tensor to numpy array first
            reference = [english_vocab.get_itos()[idx] for idx in reference_indices]

            # Convert the source tensor back to a string for translation
            source_sentence = ' '.join([arabic_vocab.get_itos()[idx.item()] for idx in src_batch[i] if idx != src_pad_idx])

            # Generate a translation using the loaded model
            translation_indices = numericalize(translate_sentence(model, source_sentence, arabic_vocab, english_vocab, device), tokenize_eng, english_vocab)
            translation = [english_vocab.get_itos()[idx] for idx in translation_indices]

            # Calculate BLEU score for the current sample
            score = bleu_score([reference], translation)

            # Accumulate the scores
            total_score += score
            total_samples += 1

    # Return the average score
    return total_score / total_samples if total_samples > 0 else 0


In [38]:
# def bleu(model, test_data, arabic_vocab, english_vocab, device):
#     total_score = 0
#     tot
#     for i in range(len(test_data)):
#         # Debugging: Check that `test_data[i].trg` and `test_data[i].src` are accessible
#         print(f"\nProcessing sentence {i+1}/{len(test_data)}")

#         # Get the target translation for the current sample
#         print(f"Target Translation:{test_data[i]['english']}")
#         reference_indices = numericalize(test_data[i]["english"], tokenize_eng, english_vocab)
#         reference = [english_vocab.get_itos()[idx] for idx in reference_indices]

#         # Generate a translation using the loaded model
#         print(f"Generated Translation:{test_data[i]['arabic']}")
#         translation_indices = numericalize(translate_sentence(model, test_data[i]["arabic"], english_vocab, arabic_vocab, device), tokenize_eng, english_vocab)
#         translation = [english_vocab.get_itos()[idx] for idx in translation_indices]

#         # Calculate BLEU score for the current sample
#         score = bleu_score([reference], translation)

#         # Accumulate the scores
#         total_score += score

#     # Return the average score
#     return total_score / len(test_data) if len(test_data) > 0 else 0


In [91]:
# Example for loading a checkpoint
def load_checkpoint(model, optimizer, path="checkpoint.pth"):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    epoch = checkpoint['epoch']
    print(f"Checkpoint loaded: Epoch {epoch}")
    return epoch

## Setting up Training Phase

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
load_model = True
save_model = True

# Training Hyperparameters
num_epochs = 100
learning_rate = 3e-4
batch_size = 256

# Model Hyperparameters
src_vocab_size = len(arabic_vocab)
trg_vocab_size = len(english_vocab)
embedding_size = 256
num_heads = 8
num_encoder_layers = 3 #decrease layers to reduce model complexity and increase speed 
num_decoder_layers = 3
dropout = 0.1
max_len = 100 # sentence length; have to either delete sentences longer than 100 or increase max length
forward_expansion = 2 # determines the size of the hidden layers for the forward pass/increases or decreases model's hidden layer size
src_pad_idx = arabic_vocab["<pad>"] # Get <pad> token indices from both vocabs
trg_pad_idx = english_vocab["<pad>"]

writer = SummaryWriter("runs/loss_plot")
step = 0

# train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
#     (train_data, valid_data, test_data),
#     batch_size=batch_size,
#     sort_within_batch=True,
#     sort_key=lambda x: len(x.src),
#     device=device,
# )
# Loading large data in manageable batch sizes
train_iterator = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn = collate_fn) # Shuffle is on so at the start of each epoch the data is shuffled for learning
valid_iterator = DataLoader(valid_dataset, batch_size=32, shuffle=False, collate_fn = collate_fn)
test_iterator = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn = collate_fn)

model = Transformer(
    embedding_size,
    src_vocab_size,
    trg_vocab_size,
    src_pad_idx,
    num_heads,
    num_encoder_layers,
    num_decoder_layers,
    forward_expansion,
    dropout,
    max_len,
    device,
).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss(ignore_index=trg_pad_idx)

if load_model:
     epoch = load_checkpoint(model, optimizer, path="my_checkpoint.pth")
     
sentence = "وَهُنَّ يَشْفَعْنَ إِلَيْهِ" #"And they will intercede for him."


for epoch in range(num_epochs):
    print(f"[Epoch {epoch} / {num_epochs}]")

    if save_model:
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "epoch": epoch
        }
        torch.save(checkpoint, "my_checkpoint.pth")

    # # model.eval()
    # translated_sentence = translate_sentence(
    #     model, sentence, arabic_vocab, english_vocab, device, max_length=100
    # )

    # print(f"Translated example sentence: \n {translated_sentence}")
    model.train()

    for i, batch in enumerate(train_iterator):
        inp_data = batch.src.to(device) # get source batch from train iterator
        target = batch.trg.to(device) # get target batch from train iterator

        # Replace out-of-range indices with UNK_INDEX
        unk_index = arabic_vocab["<unk>"]
        unk_index_eng = english_vocab["<unk>"]
        # Get the maximum index in inp_data
        max_src_index = inp_data.max().item()
        max_trg_index = target[:-1].max().item()
        # Check if the maximum index is out of range
        if max_src_index >= src_vocab_size:
            print(f"Out-of-range indices detected. Max index: {max_src_index}, Replacing with <unk> index.")
            # Replace out-of-range indices with the UNK index
            inp_data = torch.where(inp_data >= src_vocab_size, torch.tensor(unk_index, device=device), inp_data)
        # Check if the maximum index is out of range
        if max_trg_index >= trg_vocab_size:
            print(f"Out-of-range indices detected. Max index: {max_trg_index}, Replacing with <unk> index.")
            # Replace out-of-range indices with the UNK index
            target = torch.where(target >= trg_vocab_size, torch.tensor(unk_index_eng, device=device), target)

        if inp_data.max() >= src_vocab_size:
            print(f"Input data index out of range: {inp_data.max()}")
        if target.max() >= trg_vocab_size:
            print(f"Target data index out of range: {target.max()}")
        else:
          # forward propagation
          output = model(inp_data, target[:-1]) # pass in the input data(arabic) and target data(english) through the model and remove the last token in english
        #   if i % 10 == 0:
        #         # Get the actual output of the model
        #     test_output = output.argmax(2)

        #     # Use teacher forcing to generate the source and target sentences
        #     src_sentence = " ".join([arabic_vocab.get_itos()[idx] for idx in inp_data.cpu().numpy()[0]])
        #     trg_sentence = " ".join([english_vocab.get_itos()[idx] for idx in target.cpu().numpy()[0]])

        #     # Use the actual output of the model to generate the translated sentence
        #     translated_sentence = " ".join([english_vocab.get_itos()[idx] for idx in test_output.cpu().numpy()[0]])

        #     print(f"Source sentence: {src_sentence}")
        #     print(f"Target sentence: {trg_sentence}")
        #     print(f"Translated sentence: {translated_sentence}")
          output = output.reshape(-1, output.shape[2])
          target = target[1:].reshape(-1)
          optimizer.zero_grad()

          loss = loss_fn(output, target)
          loss.backward()

          torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

          optimizer.step()
          # Print loss every 10 training examples
          if i % 100 == 0:
            print(f"Loss at iteration {i}: {loss.item()}")
          writer.add_scalar("Training loss", loss, global_step=step)
          step += 1
#           score = bleu(model, test_iterator, arabic_vocab, english_vocab, device)
#           print(f"Bleu score: {score}")
# print(f"Bleu score {score*100:.2f}")


Checkpoint loaded: Epoch 32
[Epoch 0 / 100]
Loss at iteration 0: 2.739516496658325


* TRAINING PROCESS: Involves defining several key hyperparameters, including a learning rate of 3e-4, a batch size of 32 and a total of five epochs. The loss funciton used is Cross entropy loss whihc ignores padding tokens to ensure accurate evaluation. During each iteration of the training loop, the model processes batches of data, performs forward passes to generate predictions, calculates the lossses and updataes the model weights through backpropagation. This is the typical training process used by various examples and tutorials.


* EVALUATION: Use metrics like BLEU score to measure the quality of the translations produced by the model. Depending on the results, potential improvements invovle fine tuning the hyperparameters or implementing data augmentation techniques to enrich the training data.
